<a href="https://developer.apple.com/library/mac/documentation/Darwin/Reference/ManPages/man5/plist.5.html">PList manual page</a>


In [1]:
cd ~/Documents/Developpement/gist/

/Users/manu/Documents/Developpement/gist


In [2]:
import SafariBookmarkEditor
from os.path import (expanduser, isfile)

In [3]:
_path = expanduser('/tmp/BookmarksCorrupt.plist')

_path

_path = expanduser('/tmp/Bookmarks.plist')
if not isfile(_path):
    print("Bookmarks.plist doesn't appear to exist."
          "Generating new Bookmarks.plist.")

In [4]:
import plistlib
from plistlib import InvalidFileException

In [5]:
bmks = SafariBookmarkEditor.SafariBookmarks()

In [6]:
bmks.read()

In [7]:
bmks.titles

['p',
 'MaDoc',
 'Le Monde',
 'Zeit',
 'Arte',
 'France Culture',
 'Wetter Stuttgart',
 'Léo\xa0F-D',
 'FRITZ!Box']

In [8]:
import datetime
import json

def datetime_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    raise TypeError("Unknown type")

data = json.dumps(bmks.plist, default=datetime_handler)

In [25]:
!ls

SafariBookmarkEditor.py    SafariBookmarksViewer.html
SafariBookmarks.json       __pycache__


In [27]:
with open('SafariBookmarks.json', 'w') as outfile:
    json.dump(bmks.plist, outfile, default=datetime_handler)

In [14]:
from IPython.display import Javascript

In [19]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    },
});

<IPython.core.display.Javascript object>

In [20]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("660px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
  var data = convertPlotData(window.headwayVsRidership);
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  var xScale = d3.scale.linear().range([0, width]);
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
  var xMap = function(d) { return xScale(xVals(d));};
      
  var yScale = d3.scale.linear().range([height, 0]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);

  
  var cValue = function(d) { 
    if(d.day == "Saturday" || d.day == "Sunday"){
        return "Weekend";
    }
    return "Weekday";
  }
  var color = d3.scale.category10(); 
  
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");

  //NEW: TOOLTIP. 
  var tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
 
  svg.selectAll(".dot")
     .data(data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", xMap)
     .attr("cy", yMap)
     .style("fill", function(d) { return color(cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });
    
  var legend = svg.selectAll(".legend")
      .data(color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})

});

<IPython.core.display.Javascript object>

In [67]:
for k in bmks.plist.keys():
    print(type(bmks.plist[k]))
#    print(len(bmks.plist[k]))

<class 'str'>
<class 'list'>
<class 'str'>
<class 'str'>
<class 'int'>


In [45]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import (dirname, join)

In [42]:
root = "https://docs.python.org/3/library/index.html"
doc = urlopen(root).read()
soup = BeautifulSoup(doc, "lxml")


In [43]:
li = soup.find_all("li", class_='toctree-l1')

In [50]:
for i, l1 in enumerate(li):
    a = l1.find_next("a")
    print(i, join(dirname(root),a.attrs['href']), a.contents[0])
    
    for j, l2 in enumerate(l1.find_all("li", class_='toctree-l2')):
        a2 = l2.find_next("a")
        print("    ", j, join(dirname(root),a2.attrs['href']), a2.contents[0])
  

0 https://docs.python.org/3/library/intro.html 1. Introduction
1 https://docs.python.org/3/library/functions.html 2. Built-in Functions
2 https://docs.python.org/3/library/constants.html 3. Built-in Constants
     0 https://docs.python.org/3/library/constants.html#constants-added-by-the-site-module 3.1. Constants added by the 
3 https://docs.python.org/3/library/stdtypes.html 4. Built-in Types
     0 https://docs.python.org/3/library/stdtypes.html#truth-value-testing 4.1. Truth Value Testing
     1 https://docs.python.org/3/library/stdtypes.html#boolean-operations-and-or-not 4.2. Boolean Operations — 
     2 https://docs.python.org/3/library/stdtypes.html#comparisons 4.3. Comparisons
     3 https://docs.python.org/3/library/stdtypes.html#numeric-types-int-float-complex 4.4. Numeric Types — 
     4 https://docs.python.org/3/library/stdtypes.html#iterator-types 4.5. Iterator Types
     5 https://docs.python.org/3/library/stdtypes.html#sequence-types-list-tuple-range 4.6. Sequence Types —

'37. Undocumented Modules'